In [2]:
import numpy as np
f = open("input.dat","r")
lines = f.readlines()
lines = [line.strip() for line in lines if line.strip()!='']
f.close()
lines
sources = list(map(int,lines.pop(0).removeprefix('seeds: ').split(" ")))


lines = np.array([x for x in lines])
maps = [ [] for _ in range(7)]
idx_variable = 0
for line in lines[1:]:
    if len(line.split(':'))>1:
        idx_variable += 1
        continue
    maps[idx_variable].append(([x for x in list(map(int, line.split(" ")))]))

for dmap in maps:
    dmap = np.array(dmap)
    destination_range = dmap[:,0]
    source_range = dmap[:,1]
    steps = dmap[:,2]
    for i, s in enumerate(sources):
        for j, (start, finish) in enumerate(zip(source_range,source_range+steps)):
            if s >= start and s < finish:
                print(f'case1, seed start: {s}, in between {start}, {finish-1}', end=' ')
                actual_step = s - start
                tmp = destination_range[j]+actual_step
                sources[i] = tmp
                print(f'seed changed to {tmp}, because destination starts {destination_range[j]} is equal to {source_range[j]}')
    print()

       
print(sources)
print(min(sources))





case1, seed start: 79, in between 50, 97 seed changed to 81, because destination starts 52 is equal to 50
case1, seed start: 55, in between 50, 97 seed changed to 57, because destination starts 52 is equal to 50

case1, seed start: 14, in between 0, 14 seed changed to 53, because destination starts 39 is equal to 0
case1, seed start: 13, in between 0, 14 seed changed to 52, because destination starts 39 is equal to 0

case1, seed start: 53, in between 53, 60 seed changed to 49, because destination starts 49 is equal to 53
case1, seed start: 57, in between 53, 60 seed changed to 53, because destination starts 49 is equal to 53
case1, seed start: 52, in between 11, 52 seed changed to 41, because destination starts 0 is equal to 11

case1, seed start: 81, in between 25, 94 seed changed to 74, because destination starts 18 is equal to 25
case1, seed start: 49, in between 25, 94 seed changed to 42, because destination starts 18 is equal to 25
case1, seed start: 53, in between 25, 94 seed ch

In [29]:

import numpy as np
f = open("input.dat","r")
lines = f.readlines()
lines = [line.strip() for line in lines if line.strip()!='']
f.close()
tmpsources = list(map(int,lines.pop(0).removeprefix('seeds: ').split(" ")))
sources = tmpsources[::2]
sources_step = tmpsources[1::2]



lines = np.array([x for x in lines])
maps = [ [] for _ in range(7)]
idx_variable = 0
for line in lines[1:]:
    if len(line.split(':'))>1:
        idx_variable += 1
        continue
    maps[idx_variable].append(([x for x in list(map(int, line.split(" ")))]))

output = []

new_sources = sources
new_steps = sources_step
initial_len = sum(sources_step)
for jmap, dmap in enumerate(maps):
    dmap = np.array(dmap)
    destination_range = dmap[:,0]
    source_range = dmap[:,1]
    steps = dmap[:,2]

    sources = new_sources
    sources_step = new_steps
    zipped_pairs_ = zip(new_sources, new_steps)
    sources, sources_step = zip(*sorted(zipped_pairs_))
    # print('seeds to test', sources, sources_step)
    sources = list(sources) 
    sources_step = list(sources_step)

    new_sources = []
    new_steps = []

    zipped_pairs = zip(source_range,destination_range, steps) # sorting by the source range make it easier (in my head)
    source_range, destination_range, steps = zip(*sorted(zipped_pairs))
    source_range = np.array(source_range)
    destination_range = np.array(destination_range)
    steps = np.array(steps)


    for i, s in enumerate(sources):
        ends = s + sources_step[i]
        # print(f'seed start-end: {s}-{ends}')
        there_are_some_changes = False
        for j, (start, finish) in enumerate(zip(source_range,source_range+steps)):
            # print(f'Scanning {start}-{finish}')
            if ends <= start and j == 0:
                # print(f'nothing to do .. seed start: {s} - {ends}, in between {start}, {finish-1}')
                #nothing to be done
                new_sources.append(s)
                new_steps.append(sources_step[i])
                there_are_some_changes = True
            if s >= finish and j == len(source_range)-1:
                # print(f'bigger and at finish .. seed start: {s} - {ends}, in between {start}, {finish-1}')
                #nothing to be done
                new_sources.append(s)
                new_steps.append(sources_step[i])
                there_are_some_changes = True

            if s >= start and ends <= finish:
                # print(f'seed start: {s} - {ends}, in between {start}, {finish-1}', end=' ')
                actual_step = s - start
                tmp = destination_range[j]+actual_step
                # print(f'seed changed to {tmp}, because destination starts {destination_range[j]} and actual_step: {actual_step}')
                new_sources.append(tmp)
                new_steps.append(sources_step[i])
                there_are_some_changes = True

            if s >= start and s < finish and ends > finish:
                # print(f'seed SPLIT: {s} - {ends}, in between {start}, {finish-1}')
                actual_step = s - start
                tmp = destination_range[j]+actual_step
                new_sources.append(tmp)
                residual = ends-finish
                new_steps.append(sources_step[i] - residual)

                # print(f'seed changed to {tmp} ({destination_range[j]} + {actual_step}), because destination starts {destination_range[j]} and step: {sources_step[i] + residual} residual: {residual}')
                sources.insert(i+1, ends-residual)
                sources_step.insert(i+1, residual)
                # print(f'Adding source, step: {ends-residual}, {residual}')
                there_are_some_changes = True
                break
            if s < start and ends > start and ends <= finish:

                # print(f'OTHER seed SPLIT: {s} - {ends}, in between {start}, {finish-1}')
                new_sources.append(s)
                residual = ends-start
                new_steps.append(sources_step[i] - residual)

                sources.insert(i+1, ends-residual)
                sources_step.insert(i+1, residual)
                # print(f'Adding source, step: {ends-residual}, {residual}')
                there_are_some_changes = True
                break

        if not there_are_some_changes:
            new_sources.append(s)
            new_steps.append(sources_step[i])

        # print()
    


       
sources = new_sources
sources_step = new_steps
    # if jmap == 1:
    #     break
print(sources, sources_step)
print(min(sources))

print(initial_len)
final_len = sum(sources_step)
print(final_len, sources_step)




[664041710, 712679055, 825516882, 136096660, 206777808, 222429954, 226439822, 521152811, 558196027, 861012996, 887862361, 946120886, 1096887697, 1193511298, 1203152318, 1230040740, 4273894768, 1702450793, 1724616659, 1739593580, 1833822942, 1839010340, 1232230400, 1438612989, 1460360716, 1489132624, 1527193957, 1897060602, 1916994259, 1955881140, 2431553842, 2534128701, 2614262754, 2675077212, 2719181727, 2731120471, 2840087745, 2967053444, 3010934736, 3116585139, 3183376440, 3237293229, 3249191479, 3266889914, 3328175122, 3356068546, 3384156939, 3420588827, 3576635780, 3704588883, 3710263359, 3717108856, 3761811960, 3833664421, 3853741770, 3906017175, 3976602196, 3991368753, 4104994517, 4217167942, 4236367274, 4270406223, 1588644556, 1623045508, 1663516146, 1163080465] [48637345, 8647680, 10124827, 22081058, 15652146, 4009868, 14073366, 640035, 15883071, 15336451, 4734717, 55095155, 20050157, 9641020, 6084949, 2189660, 21072528, 22165866, 13441982, 633466, 3053835, 17012477, 76875428,

In [64]:
a = [0,1,2]
for v in a:
    print(v)
    a.append(v+3)
    if len(a)>10:
        break
a

0
1
2
3
4
5
6
7


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [63]:
# too small 55716176